In [442]:
!pip install nltk
!pip install beautifulsoup4
!pip install plotly

from bs4 import BeautifulSoup
import requests
import json
import re


In [443]:
url= "https://www.aljazeera.com/where/mozambique/"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
n = 1
articles_list = []
# print(soup.prettify()) # print the parsed data of html
for article in soup.find_all("article"):
  if n != 11:
    articles_list.append(str(article.text))   
  else:
    break
  n += 1

print(articles_list)

['Floods hit South Africa’s KwaZu\xadlu-Na\xadtal province againThe floods come only a month af\xadter tor\xadren\xadtial rains killed 435 peo\xadple in KwaZu\xadlu-Na\xadtal.Published On 23 May 202223 May 2022', 'Mozam\xadbique: Cy\xadclone Gombe death toll ris\xades to 53In re\xadcent years, South\xadern Africa has suf\xadfered re\xadpeat\xaded dev\xadas\xadtat\xading cy\xadclones that used to be rel\xada\xadtive\xadly rare.Published On 18 Mar 202218 Mar 2022', 'Mozam\xadbique an\xadnounces new prime min\xadis\xadter af\xadter cab\xadi\xadnet reshuf\xadfleNew min\xadis\xadters were named af\xadter a ma\xadjor reshuf\xadfle by Pres\xadi\xaddent Fil\xadipe Nyusi.Published On 4 Mar 20224 Mar 2022', 'Analy\xadsis: Can African gas re\xadplace Russ\xadian sup\xadplies to Eu\xadrope?A lack of in\xadfra\xadstruc\xadture, not ca\xadpac\xadi\xadty, could hurt the con\xadti\xadnent’s plan to sup\xadply gas to Eu\xadrope in Rus\xadsia’s stead.Published On 1 Mar 20221 Mar 2022', 'Dozens dead from

In [444]:
links = []
titles = []
data_title = {}
n = 1
for articles in soup.find_all("article"):
  if n != 11:
    href_data = articles.find("a")  # contains 2 rows
    links.append(href_data['href'])
    span_data = articles.find("span").text
    titles.extend(span_data)
    data_title[href_data['href']] = span_data
  else:
    break
  n += 1

print(data_title)

{'/news/2022/5/23/floods-hit-south-africas-kwazulu-natal-province-again': 'Floods hit South Africa’s KwaZu\xadlu-Na\xadtal province again', '/news/2022/3/18/mozambique-cyclone-gombe-death-toll-rises-to-53': 'Mozam\xadbique: Cy\xadclone Gombe death toll ris\xades to 53', '/news/2022/3/4/mozambique-announces-new-prime-minister-and-finance-minister': 'Mozam\xadbique an\xadnounces new prime min\xadis\xadter af\xadter cab\xadi\xadnet reshuf\xadfle', '/economy/2022/3/1/analysis-can-african-gas-replace-russian-supplies-to-europe': 'Analy\xadsis: Can African gas re\xadplace Russ\xadian sup\xadplies to Eu\xadrope?', '/news/2022/1/27/at-least-70-dead-from-tropical-storm-ana-in-southern-africa': 'Dozens dead from Trop\xadi\xadcal Storm Ana in south\xadern Africa', '/news/2022/1/12/southern-africa-bloc-sadc-extends-mozambique-mission': 'South\xadern Africa bloc SADC ex\xadtends Mozam\xadbique mis\xadsion', '/program/start-here/': ' From:  Start Here', '/news/2021/9/24/in-mozambique-kagame-says-rwa

In [445]:
data__para = {}

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

for link in links:
  url = "https://www.aljazeera.com"
  html_content_link = requests.get(url+link).text
  soup_link = BeautifulSoup(html_content_link, "lxml")
  paragraphs = soup_link.find_all("p")
  for i in range(len(paragraphs)):
    paragraphs[i] = remove_tags(str(paragraphs[i]))
  data__para[link] = paragraphs

print(data__para)

{'/news/2022/5/23/floods-hit-south-africas-kwazulu-natal-province-again': ['The floods are happening only a month after torrential rains killed 435 people in the KwaZulu Natal province.', 'Heavy rains in South Africa have forced hundreds of people to flee their homes on the east coast, according to officials, only a month after the worst floods in living memory killed more than 400.\nThe floods over the weekend hit infrastructure, roads, bridges and damaged buildings, mainly in the port city of Durban, the capital of KwaZulu-Natal province and home to some 3.9 million people.', '“Some areas are inaccessible and have become islands at this stage,” KwaZulu-Natal Premier Sihle Zikalala said during a news conference on Sunday.\nNo fatalities were immediately recorded, but Zikalala added: “We are yet to receive a full comprehensive report on the impact of these latest heavy rains, but as reports come in a worrying picture is however emerging.”\nIn April, torrential rain led to floods, lands

In [446]:
print("Number of articles: {} and Number of links and paragraphs: {}".format(len(data_title), len(data__para)))

Number of articles: 10 and Number of links and paragraphs: 10


In [447]:
input_data = {}
for link in data_title:
  if link in data__para:
    input_data[data_title[link]] = " ".join(data__para[link])

print(input_data)

{'Floods hit South Africa’s KwaZu\xadlu-Na\xadtal province again': 'The floods are happening only a month after torrential rains killed 435 people in the KwaZulu Natal province. Heavy rains in South Africa have forced hundreds of people to flee their homes on the east coast, according to officials, only a month after the worst floods in living memory killed more than 400.\nThe floods over the weekend hit infrastructure, roads, bridges and damaged buildings, mainly in the port city of Durban, the capital of KwaZulu-Natal province and home to some 3.9 million people. “Some areas are inaccessible and have become islands at this stage,” KwaZulu-Natal Premier Sihle Zikalala said during a news conference on Sunday.\nNo fatalities were immediately recorded, but Zikalala added: “We are yet to receive a full comprehensive report on the impact of these latest heavy rains, but as reports come in a worrying picture is however emerging.”\nIn April, torrential rain led to floods, landslides and the 

In [448]:
# Serializing json  
with open("data.json", "w") as outfile:
  json.dump(input_data, outfile, indent = 1) 

In [449]:
import nltk

nltk.download([    "names",
     "stopwords",
     "state_union",
     "twitter_samples",
     "movie_reviews",
     "averaged_perceptron_tagger",
     "vader_lexicon",
     "punkt",
])
nltk.download('shakespeare')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   

True

In [450]:
# input_data = temp1

In [451]:
# temp = input_data
# temp1 = temp

In [452]:
#preprocessing for paragraphs

import re

stopwords = nltk.corpus.stopwords.words("english") #remove stopwords
stopwords.extend([w.lower() for w in nltk.corpus.names.words()]) # remove common nouns 

for title, paragraph in input_data.items():
  
  paragraph = paragraph.split()

  for para in range(len(paragraph)):

#convert all to lower case
    paragraph[para] = paragraph[para].lower()

# remove urls
    paragraph[para] = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', paragraph[para], flags=re.MULTILINE)
    paragraph[para] = re.sub(r'\\[a-zA-z]', '', paragraph[para])
  
# remove html tags
    paragraph[para] = BeautifulSoup(paragraph[para]).get_text()

# remove non-english characters
    paragraph[para] = re.sub(r'[^a-zA-Z]', ' ', paragraph[para])  

# remove whitespaces
    paragraph[para] = re.sub(' +', ' ', paragraph[para])

# remove astopwords and common nouns
    if paragraph[para].lower() not in stopwords:
      paragraph[para] = paragraph[para]
    else:
      paragraph[para] = ''

# remove whitespaces
    paragraph[para] = "".join(paragraph[para].split())

  paragraph = " ".join(paragraph)
  paragraph =  " ".join(paragraph.split())
  input_data[title] = paragraph

print(input_data)

{'Floods hit South Africa’s KwaZu\xadlu-Na\xadtal province again': 'floods happening month torrential rains killed people kwazulu natal province heavy rains south africa forced hundreds people flee homes east coast according officials month worst floods living memory killed floods weekend hit infrastructure roads bridges damaged buildings mainly port city durban capital kwazulunatal province home million people some areas inaccessible become islands stage kwazulunatal premier sihle zikalala said news conference sunday fatalities immediately recorded zikalala added we yet receive full comprehensive report impact latest heavy rains reports come worrying picture however emerging april torrential rain led floods landslides death people kwazulunatal authorities said survivors left without drinking water nearly two weeks repairs still ongoing floods caused damage hundreds millions dollars sunday parts road near umdloti beach resort north durban washed away leaving gaping holes cars balanced 

In [453]:
#preprocessing for articles

input_data_temp = {}
import re

stopwords = nltk.corpus.stopwords.words("english") #remove stopwords
stopwords.extend([w.lower() for w in nltk.corpus.names.words()]) # remove common nouns 

for title, paragraph in input_data.items():
  
  title = title.split()

  for para in range(len(title)):

#convert all to lower case
    title[para] = title[para].lower()

# remove urls
    title[para] = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', title[para], flags=re.MULTILINE)
    title[para] = re.sub(r'\\[a-zA-z]', '', title[para])
  
# remove html tags
    title[para] = BeautifulSoup(title[para]).get_text()

# remove non-english characters
    title[para] = re.sub(r'[^a-zA-Z]', ' ', title[para])  

# remove whitespaces
    title[para] = re.sub(' +', ' ', title[para])

# # remove astopwords and common nouns
#     if title[para].lower() not in stopwords:
#       title[para] = title[para]
#     else:
#       title[para] = ''

# remove whitespaces
    title[para] = "".join(title[para].split())

  title = " ".join(title)
  title =  " ".join(title.split())
  input_data_temp[title] = paragraph

input_data = input_data_temp
print(input_data)

{'floods hit south africas kwazulunatal province again': 'floods happening month torrential rains killed people kwazulu natal province heavy rains south africa forced hundreds people flee homes east coast according officials month worst floods living memory killed floods weekend hit infrastructure roads bridges damaged buildings mainly port city durban capital kwazulunatal province home million people some areas inaccessible become islands stage kwazulunatal premier sihle zikalala said news conference sunday fatalities immediately recorded zikalala added we yet receive full comprehensive report impact latest heavy rains reports come worrying picture however emerging april torrential rain led floods landslides death people kwazulunatal authorities said survivors left without drinking water nearly two weeks repairs still ongoing floods caused damage hundreds millions dollars sunday parts road near umdloti beach resort north durban washed away leaving gaping holes cars balanced edge afp n

In [467]:
# scores based on article paragraphs

from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

for article_title, article_paragraph in input_data.items():
  print("The score for \"{}\".... article paragraph is: {}, \n".format(" ".join(article_paragraph.split()[:10]), sia.polarity_scores(article_paragraph)))

The score for "floods happening month torrential rains killed people kwazulu natal province".... article paragraph is: {'neg': 0.164, 'neu': 0.768, 'pos': 0.068, 'compound': -0.9816}, 

The score for "recent years southern africa suffered repeated devastating cyclones scientists believe".... article paragraph is: {'neg': 0.346, 'neu': 0.642, 'pos': 0.012, 'compound': -0.9933}, 

The score for "new ministers named reshuffle president nyusi mozambique president nyusi appointed".... article paragraph is: {'neg': 0.073, 'neu': 0.87, 'pos': 0.057, 'compound': -0.304}, 

The score for "lack infrastructure capacity could hurt continents plan supply gas europe".... article paragraph is: {'neg': 0.054, 'neu': 0.822, 'pos': 0.123, 'compound': 0.9913}, 

The score for "least people killed struck madagascar mozambique malawi authorities scrambling repair".... article paragraph is: {'neg': 0.297, 'neu': 0.638, 'pos': 0.065, 'compound': -0.9957}, 

The score for "regional body says progress made reb

In [455]:
# scores based on article titles

# scores based on article paragraphs

from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

for article_title, article_para in input_data.items():
  print("The score for \"{}\" article is: {}, \n".format(article_title, sia.polarity_scores(article_title)))

The score for "floods hit south africas kwazulunatal province again" article is: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 

The score for "mozambique cyclone gombe death toll rises to" article is: {'neg': 0.394, 'neu': 0.606, 'pos': 0.0, 'compound': -0.5994}, 

The score for "mozambique announces new prime minister after cabinet reshuffle" article is: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 

The score for "analysis can african gas replace russian supplies to europe" article is: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 

The score for "dozens dead from tropical storm ana in southern africa" article is: {'neg': 0.35, 'neu': 0.65, 'pos': 0.0, 'compound': -0.6486}, 

The score for "southern africa bloc sadc extends mozambique mission" article is: {'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.128}, 

The score for "from start here" article is: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 

The score for "in mozambique kagame says rw

In [468]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint

sia = SIA()
results = []

for title, paragraph in input_data.items():
    pol_score = sia.polarity_scores(paragraph)
    pol_score['headline'] = title
    results.append(pol_score)

pprint(results[:], width=100)

# Graph

import plotly.express as px
import pandas
df = pd.DataFrame.from_dict(results)
df.drop("compound", axis = 1, inplace = True)
df
fig = px.histogram(df,
                   x='headline',
                   title='Sentiment Score of News Articles by PARAGRAPH',
                   y= ['neg', 'neu','pos'],
                   barmode='group',
                  color_discrete_sequence=["red", "blue", "green"])
fig.update_xaxes( title='Article Title').update_yaxes(title='Sentiment score')
fig.show()

[{'compound': -0.9816,
  'headline': 'floods hit south africas kwazulunatal province again',
  'neg': 0.164,
  'neu': 0.768,
  'pos': 0.068},
 {'compound': -0.9933,
  'headline': 'mozambique cyclone gombe death toll rises to',
  'neg': 0.346,
  'neu': 0.642,
  'pos': 0.012},
 {'compound': -0.304,
  'headline': 'mozambique announces new prime minister after cabinet reshuffle',
  'neg': 0.073,
  'neu': 0.87,
  'pos': 0.057},
 {'compound': 0.9913,
  'headline': 'analysis can african gas replace russian supplies to europe',
  'neg': 0.054,
  'neu': 0.822,
  'pos': 0.123},
 {'compound': -0.9957,
  'headline': 'dozens dead from tropical storm ana in southern africa',
  'neg': 0.297,
  'neu': 0.638,
  'pos': 0.065},
 {'compound': 0.946,
  'headline': 'southern africa bloc sadc extends mozambique mission',
  'neg': 0.069,
  'neu': 0.76,
  'pos': 0.171},
 {'compound': -0.9873, 'headline': 'from start here', 'neg': 0.329, 'neu': 0.63, 'pos': 0.04},
 {'compound': -0.9887,
  'headline': 'in mozamb

In [469]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint

sia = SIA()
results = []

for title, paragraph in input_data.items():
    pol_score = sia.polarity_scores(title)
    pol_score['headline'] = title
    results.append(pol_score)

pprint(results[:], width=100)

import plotly.express as px
import pandas
df = pd.DataFrame.from_dict(results)
df.drop("compound", axis = 1, inplace = True)
df
fig = px.histogram(df,
                   x='headline',
                   title='Sentiment Score of News Articles by TITLE',
                   y= ['neg', 'neu','pos'],
                   barmode='group',
                  color_discrete_sequence=["red", "blue", "green"])
fig.update_xaxes( title='Article Title').update_yaxes(title='Sentiment score')
fig.show()


[{'compound': 0.0,
  'headline': 'floods hit south africas kwazulunatal province again',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0},
 {'compound': -0.5994,
  'headline': 'mozambique cyclone gombe death toll rises to',
  'neg': 0.394,
  'neu': 0.606,
  'pos': 0.0},
 {'compound': 0.0,
  'headline': 'mozambique announces new prime minister after cabinet reshuffle',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0},
 {'compound': 0.0,
  'headline': 'analysis can african gas replace russian supplies to europe',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0},
 {'compound': -0.6486,
  'headline': 'dozens dead from tropical storm ana in southern africa',
  'neg': 0.35,
  'neu': 0.65,
  'pos': 0.0},
 {'compound': 0.128,
  'headline': 'southern africa bloc sadc extends mozambique mission',
  'neg': 0.0,
  'neu': 0.8,
  'pos': 0.2},
 {'compound': 0.0, 'headline': 'from start here', 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': 0.0,
  'headline': 'in mozambique kagame says rwandan troops work not over',
  